In [1]:
from dotenv import load_dotenv
import yaml
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from crewai.tools import tool
from IPython.display import display, Markdown
import requests

load_dotenv()

/Users/raj_attune/Code/crewAI-course-examples/web_research_rag/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


True

In [2]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [3]:
synth_llm_small = LLM(
    model="anthropic/claude-3-5-sonnet-20241022",
    temperature=0.2
)
synth_llm_large = LLM(
    model="anthropic/claude-3-5-sonnet-20241022",
    temperature=0.2
)

In [4]:
finhealth_sales_analyst_agent = Agent(
    config=agents_config['finhealth_sales_analyst'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=synth_llm_small,
    max_rpm=5,
    respect_context_window=True
)
chief_sales_analyst_agent = Agent(
    config=agents_config['chief_sales_analyst'],
    llm=synth_llm_large,
    max_rpm=25,
    respect_context_window=True
)

In [5]:
research_tasks = [
    Task(config=tasks_config[tc], agent=finhealth_sales_analyst_agent, async_execution=True) \
    for tc in list(tasks_config.keys()) \
    if tc.startswith('finhealth_research')
]

finhealth_report_task = Task(
   config=tasks_config['finhealth_report_task'],
    agent=chief_sales_analyst_agent,
    context=research_tasks
)

In [6]:
finhealth_crew = Crew(
    agents=[finhealth_sales_analyst_agent],
    tasks=[
        *research_tasks,
        finhealth_report_task
    ],
    verbose=True
)

In [7]:
company = 'Golden 1 Credit Union'
result = finhealth_crew.kickoff(inputs={
  'Company': company
})

# Agent: Lead Sales Analyst - Financial Health and Wellness
## Task: Conduct a thorough research about Golden 1 Credit Union to evaluate their financial health and wellness budget.  Do a deep search on the company website, including its about page, annual reports, press releases, board statements and other information on its website showing its financial health and wellness budget.  Prioritize information in the last 2-3 years, given that the current year is 2025.  Focus your evaluation of Golden 1 Credit Union on the following observable criteria, in order of decreasing importance: - Job postings for financial health-related positions - Sponsorship of financial health events/programs - Grant programs supporting financial health initiatives - Published case studies about financial health investments - Partnership announcements with finhealth vendors/organizations - Launch of new financial health products/services Do not make up any information that you do not find in your search.Do not

In [10]:
with open('output/%s.md'%company, 'w') as file:
    file.write(str(result))

In [9]:
for n in range(len(research_tasks)):
    _task = research_tasks[n]
    _crew = Crew(
        agents=[finhealth_sales_analyst_agent],
        tasks=[_task],
        verbose=True
    )
    _result = _crew.kickoff(inputs={
        'Company': company
    })
    with open('output/%s - %d.md' % (company,n+1), 'w') as file:
        file.write(str(_result))


2025-02-14 14:22:45,135 - 8599255104 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Lead Sales Analyst - Financial Health and Wellness
## Task: Conduct a thorough research about Golden 1 Credit Union to evaluate their board commitment to financial health and wellness. Do a deep search on the company website, including its about page, annual reports, press releases, board statements and other information on its website showing the committment of their board to financial health and wellness. Prioritize information in the \ last 2-3 years, given that the current year is 2025. Only use information that  you found in your search.  Focus your evaluation of Golden 1 Credit Union on the following observable criteria,  in order of decreasing importance: - Published strategic plans or annual reports mentioning financial health initiatives - Public statements from board members about financial health priorities - Dedicated financial health sections on website/marketing materials - Board meeting minutes (if public) discussing financial health programs - Financial health 

2025-02-14 14:23:22,061 - 8599255104 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Lead Sales Analyst - Financial Health and Wellness
## Final Answer: 
Board Commitment to Financial Health and Wellness Evaluation for Golden 1 Credit Union

Overall Rating: 4/5 (Strong Commitment)

Detailed Evaluation:

1. Published Strategic Plans and Initiatives:
- Launched Enrich™ Financial Wellness Platform in May 2023 in partnership with iGrad
- Platform provides personalized financial education with key features:
  * Your Money Personality™ financial behavior assessment
  * Personalized financial education courses
  * Retirement and home affordability analyzers
  * Student loan and higher education tools
- Demonstrated measurable impact from platform:
  * 35% increase in users on track with savings goals
  * 14% increase in retirement savings contributions
  * 59% increase in emergency savings
  * 28% increase in full credit card debt payment

Sources:
- https://enrich.org/golden-1-credit-union-teams-up-with-igrad-to-offer-the-enrich-personalized-financial-wellness-pro

2025-02-14 14:23:51,827 - 8599255104 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


...



# Agent: Lead Sales Analyst - Financial Health and Wellness
## Final Answer: 
Golden 1 Credit Union - Financial Health and Wellness Evaluation Report

Overall Rating: 4/5 (Strong Evidence of Commitment to Financial Health and Wellness)

1. Member-Facing Financial Health Resources and Tools (Strongest Evidence)
- Launched Enrich™ Personalized Financial Wellness Program in partnership with iGrad (2023)
  - Digital platform replacing previous Learning Lab
  - Offers comprehensive financial education courses
  - Provides personalized action plans on topics including:
    * Budgeting
    * Mortgages
    * Healthcare
    * College savings
    * Student loan management
- Financial Wellness Center with free resources to:
  * Learn money management skills
  * Reduce financial stress
  * Improve credit
  * Gain financial peace of mind

Sources:
- https://enrich.org/golden-1-credit-union-teams-up-with-igrad-to-offer-the-enrich-personalized-financial-wellness-program-to-its-members/
- https://ww

2025-02-14 14:27:48,230 - 8599255104 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Lead Sales Analyst - Financial Health and Wellness
## Final Answer: 
# Golden 1 Credit Union - Innovation Adoption Report

## Overall Innovation Adoption Rating: 4/5

### Detailed Evaluation of Innovation Initiatives

#### 1. Implementation of Emerging Fintech Solutions
- **NCR Voyix Digital Banking Partnership (June 2024)**
  - Selected NCR Voyix Digital-First Banking solution to drive digital transformation
  - Demonstrates commitment to cutting-edge digital banking technologies
  - Sources: 
    - NCR Investor Relations
    - Yahoo Finance
    - Candescent Insights

- **Prizeout Partners CUSO Collaboration**
  - Joined Prizeout Partners, a Credit Union Service Organization (CUSO)
  - Implemented innovative technology for member savings opportunities
  - Expanded digital revenue streams
  - Sources:
    - CU Times
    - Globe Newswire
    - CB Insights

#### 2. Beta Testing Partnerships with Technology Providers
- Actively engaged with NCR Voyix for digital banking transfo

2025-02-14 14:28:18,778 - 8599255104 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Lead Sales Analyst - Financial Health and Wellness
## Final Answer: 
Industry Leadership Report for Golden 1 Credit Union (2024)

OVERALL RATING: 4/5 ⭐⭐⭐⭐

Detailed Evaluation of Industry Leadership:

1. Industry Awards and Recognition
- Received 7 prestigious national recognitions in 2024
- Key Awards:
  a) American Banker - Best Credit Unions to Work For (4th consecutive year)
  b) Forbes - America's Best-In-State Employers 2024 (5th time)
  c) Forbes - America's Best Employers for Women 2024
  d) Newsweek - America's Greatest Workplaces for Women 2024
  e) Newsweek - America's Greatest Workplaces 2024
  f) Forbes - America's Best-In-State Credit Unions 2024
  g) TIME Magazine - America's Best Mid-Size Companies 2024

Sources:
- [Golden 1 Press Release: National Accolades 2024](https://www.golden1.com/discoverg1/press-releases/2024/Golden%201%20Credit%20Union%20Honored%20with%20Multiple%20National%20Accolades)

2. Speaking Slots and Conference Participation
- Limited publi

In [9]:
display(Markdown(str(result)))

# Golden 1 Credit Union Financial Health Maturity Assessment
## Comprehensive Synthesis Report

### Introduction
Golden 1 Credit Union, with over $19 billion in assets and serving more than 1 million members throughout California, demonstrates a strong commitment to financial health and wellness through comprehensive programs, strategic partnerships, and innovative solutions.

### Overall Financial Health Maturity Rating: 4/5
This rating reflects strong evidence of commitment and maturity across multiple dimensions of financial health integration.

### Key Dimensions Analysis

#### 1. Strategic Commitment to Financial Health (4/5)
- Comprehensive Financial Wellness Center (https://www.golden1.com/financial-wellness)
- Partnership with iGrad for Enrich™ Personalized Financial Wellness Program
- First-ever Financial Resource Center in Del Paso Heights (2024)
- $1 Million awarded to 42 local organizations (2025)

#### 2. Leadership and Governance (3.5/5)
- CEO Donna Bland recognized in American Banker's "Most Powerful Women in Credit Unions" (2022, 2024)
- Member of Financial Health Network
- Limited evidence of executive speaking engagements
- Strong community impact initiatives

#### 3. Program Implementation and Resources (4.5/5)
- Comprehensive digital platform featuring:
  - Personalized financial education courses
  - Action plans for budgeting, mortgages, healthcare
  - Student loan management tools
  - Credit monitoring services
- SpringFour partnership providing affordable resources
- Financial Wellness Challenge sessions (1,600+ participants)

#### 4. Operational Integration (4/5)
- Financial health metrics integrated into product marketing
- Staff training programs
- Digital banking features supporting financial wellness
- 83% employee satisfaction rate

#### 5. Community Impact and Recognition (4.5/5)
- Nearly $600K annual nonprofit contributions
- Approximately 500 college scholarships
- Multiple national accolades including:
  - Fast Company's "Brands That Matter" (2023)
  - TIME's America's Best Mid-Size Companies (2024)
  - Forbes Best-In-State Employer

### Strengths
1. Comprehensive digital financial wellness platform
2. Strong community engagement and impact
3. Innovative partnerships (iGrad, SpringFour)
4. Multiple national recognitions
5. Substantial financial commitment to initiatives

### Areas for Development
1. Limited evidence of executive speaking engagements
2. Could expand research publications on financial health impact
3. Opportunity for more transparent documentation of executive involvement
4. Potential to increase member success stories visibility

### Recommendations for Stakeholders

#### Sales Approach
1. Lead with the comprehensive digital financial wellness platform
2. Emphasize community impact and commitment
3. Highlight national recognition and awards
4. Focus on innovative partnerships and solutions

#### Partnership Opportunities
1. Explore speaking engagement collaborations
2. Consider joint research initiatives
3. Investigate financial education program partnerships
4. Explore community impact expansion opportunities

### Sources
All findings are supported by multiple sources including:
- https://www.golden1.com/financial-wellness
- https://www.golden1.com/financial-resource-center
- https://finhealthnetwork.org/author/companies/golden-1-credit-union/
- https://www.golden1.com/our-community/our-impact
- https://enrich.org/golden-1-credit-union-teams-up-with-igrad
- https://www.golden1.com/discoverg1/press-releases/

### Conclusion
Golden 1 Credit Union demonstrates a mature, well-developed approach to financial health and wellness, positioning itself as a leader in the space. Their comprehensive strategy, substantial resource commitment, and innovative solutions provide strong evidence of their dedication to improving members' financial well-being. While there are areas for potential growth, the overall financial health maturity level is strong and continues to evolve positively.